# Cutoff Power-Law or Comptonized Power-Law

This program shows you how to calcualte the integrated photon and energy fluxes of the COPL using two different methods.

1.  Integrating the Function in Wolfram Alpha and then passing an energy array to the integrated function.
2.  Using scipy.integrate to numerically integrate the function. This way is more precise.

### The equation for the Cutoff Power Law can be found here:
https://heasarc.gsfc.nasa.gov/xanadu/xspec/manual/XSmodelCutoffpl.html

#### \** NOTE:  I switch the sign on the cutoff power-law index.

Original Function, before integration: 
    
\begin{equation}
f\left(E\right) = N \times \left(\frac{E}{100}\right)^{\alpha} \times \exp \left( -\frac{E}{E_{C}} \right)  dE \ \ \ \ \ \ \ \         
\end{equation}

where $\alpha$ is the cutoff power-law index or low-energy slope.

$E_C$ is the high energy cutoff.

### My version of this equation:
I use different parameters to represent the variables that can be used within python to run calculations.

Original Function, before integration: 
    
    cplIndex or alpha: low energy index
    cutoff:  high-energy cutoff
    N:     normalization
    E:     energy to integrate over, 10 to 10,000 keV


    f(E) = N * (E**(cplIdx) * exp(-E/cutoff)) dE

### Converstion between High Energy Cutoff and Epeak energy:
Sometimes you'll see the energy as Epeak.

    epk    = 580.22058
    alpha  = -1.0534484

    ecut   = (epk)/(alpha + 2.); ecut
    ecut   = (580.22058)/(-1.0534484 + 2.)
    # ecut = 612.9835711016706

\begin{equation}
E_{pk} = E_{cutoff} \times \left( \alpha + 2.0 \right) \ \ \ \ \ 
\end{equation}

# Begin Program

In [1]:
from __future__ import division
import numpy as np
from scipy import integrate

### Constants and Parameters

In [2]:
keVtoerg    = 1.60217657E-9
emin        = 10.0
emax        = 10000.0

pars     = [-1.0534484, 612.9835711016706, 0.017022533]  
# [cplIndex, highEcut, norm]

def get_parVals():
    pars     = [-1.0534484, 612.9835711016706, 0.017022533]  
    return pars


## Wolfram Alpha Integration of the Cutoff Power-Law:

http://www.wolframalpha.com/input/?i=((x%2F100)%5Ea+*+exp(-x%2FC)+)+dx


When using Wolfram Alpha to integrate, be careful which letters you use for parameters.  Wolfram alpha has some letters set aside to mean something.  If they are used, you will not get the right answer. For example, E stands for exponential. Do NOT use E for energy.

N can be left out of integration. Simply multiply it back on at the end. The more parameters you have, the less likely Wolfram Alpha will calculate the function without a calculation time issue.

    a - cutoff power-law index
    C - high energy cutoff
    N - normalization
    x - energy


In [3]:
'''
See here to understand the difference between the regularized (normalized)
and generalized (non-normalized) incomplete gamma functions.  
http://www.boost.org/doc/libs/1_61_0/libs/math/doc/html/math_toolkit/sf_gamma/igamma.html

scipy.special's version of the incomplete gamma function is the 
** regularized ** version and doesn't work well here.
Read about it here:
https://docs.scipy.org/doc/scipy/reference/generated/scipy.special.gammainc.html

The mpmath version is the ** generalized **  version of the 
incomplete gamma function and it does work well here.
Please read about it here.
http://mpmath.org/doc/0.19/functions/expintegrals.html


   Evaluation of a generalized incomplete gamma function:
fp.gammainc( cplIdx + 1., a = (engs[i]/cutoff) )
a - the lower integral limit.
b - the upper integral limit, default: b is infinity.
'''

def copl(engs, flux, *params):
    '''
    I do not use a here for the cutoff power-law index (or alpha) because of the 
    a used in the lower limit of the gammainc calculation.  I didn't want them 
    to become confused.
    
    '''
    from mpmath import gammainc
    from mpmath import fp
    import time
    
    start_time = time.time()
    
    for i in range(len(engs)-1):
        cplIdx   = float(params[0])    # low-energy slope (alpha)
        cutoff   = float(params[1])    # cutoff energy
        lowIntegral   = ((-100.0**(-cplIdx))*(cutoff**(cplIdx+1.))) * float(fp.gammainc( cplIdx + 1., a=(engs[i]/cutoff)) ) 
        highIntegral  = ((-100.0**(-cplIdx))*(cutoff**(cplIdx+1.))) * float(fp.gammainc( cplIdx + 1., a=(engs[i+1]/cutoff)) )
        val = (highIntegral - lowIntegral)
        flux[i]       = val
    stop_time = time.time() - start_time 
    print('time: %f seconds'%(stop_time))

#### We ran this function twice:
Once with fp.gammainc and once without it, gammainc.

fp stands for fast low-precision arithmetic.  This speeds up the process.

    With fp:     2.908276 seconds  
        Fluxes: 6.167920487 and 1.535317095e-06
        
    Without fp:  11.440344 seconds
        Fluxes: 6.167920487 and 1.535317095e-06
        
You can see that the lower precision doesn't change the flux values.
http://docs.sympy.org/0.6.7/modules/mpmath/basics.html

In [4]:
N      = 5000
engs   = np.logspace(1, 4, N)
flux   = np.zeros(N)

# WILL STORE CALCULATIONS IN FLUX ARRAY.
copl(engs, flux, *pars)

norm   = pars[-1]
flux_ph = np.sum(flux) * norm

# NO NEED TO MAKE AN ESBPL FUNCTION.  MULTIPLY ENGS BY THE FLUX.
flux_en = np.sum(flux * engs * keVtoerg) * norm

print(
'''
Photon Flux:  %.9f \t photons s^-1 cm^-2
Energy Flux:  %.9e \t ergs s^-1 cm^-2
'''%(flux_ph, flux_en))

time: 8.073252 seconds

Photon Flux:  6.167920487 	 photons s^-1 cm^-2
Energy Flux:  1.535317095e-06 	 ergs s^-1 cm^-2



## FUNCTION WITHOUT BEING INTEGRATED.

In [5]:
def copl(energy):
    from numpy import exp
    cplIndex, highEcut, norm = get_parVals()
    a = float(cplIndex)
    C = float(highEcut)
    N = float(norm)
    eng = energy
    return N * ((eng/100.0)**a) * (exp(-eng/C))

def ecopl(energy):
    eng = energy
    return eng * copl(eng)

In [6]:
Flux_Ph = integrate.quad(copl, emin, emax, limit=100)[0]
Flux_En = integrate.quad(ecopl, emin, emax, limit=100)[0] * keVtoerg

print(
'''
Photon Flux:  %.9f \t photons s^-1 cm^-2
Energy Flux:  %.9e \t ergs s^-1 cm^-2
'''%(Flux_Ph, Flux_En))


Photon Flux:  6.167920487 	 photons s^-1 cm^-2
Energy Flux:  1.536378106e-06 	 ergs s^-1 cm^-2

